In [25]:
import numpy as np
import pandas as pd
from typing import Any, cast, Dict

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
from sklearn.preprocessing import RobustScaler
from read_data import read_datasets
from data_cleaning import DataCleaner, prep_dataframe

dep_var = "Log(Rmax)"
# dep_var = "Log(Efficiency)"
use_crossval = True

all_data = read_datasets()
combined_data = prep_dataframe(all_data, dep_var)

cleaner = DataCleaner(RobustScaler(), dep_var)
data = cleaner.fit_transform(combined_data)


Unknown processor: 'NEC', full name: 'NEC  3.200GHz' @ Earth Simulator, 2009
Unknown processor: 'NEC', full name: 'NEC  3.200GHz' @ Earth Simulator, 2009
Unknown processor: 'NEC', full name: 'NEC  3.200GHz' @ Earth Simulator, 2009
Unknown processor: 'NEC', full name: 'NEC  3.200GHz' @ Earth Simulator, 2009
Unknown processor: 'NEC', full name: 'NEC  3.20GHz' @ Earth Simulator, 2009
Unknown processor: 'Xeon EM64T', full name: 'Xeon EM64T  3.60GHz' @ Thunderbird, 2006
Filtered duplicates to go from 10000 rows to 2476


## System Types

See how many of each type of system there are and how they have changed over time


In [34]:
print(combined_data.Architecture.value_counts())

# So only 1 constellation, which is it?
combined_data[combined_data.Architecture == 'Constellations']


Cluster           8688
MPP               1311
Constellations       1
Name: Architecture, dtype: int64


,Architecture,Microarchitecture,Year,Processor Speed (MHz),Total Cores,Co-Processor Cores to Total Cores,Date,Log(Rmax)
7958,Constellations,Montecito,2006,1600,9968,0.0,201111,3.967


In [35]:
# How about after we do things like drop duplicates
num_const = data.x0_Constellations.value_counts()[1]
print("Constellations: ", num_const)
num_mpp = data.x0_MPP.value_counts()[1]
print("Cluster: ", num_mpp)
num_cluster = len(data) - num_const - num_mpp
print("Cluster: ", num_cluster)

# So only 1 constellation, which is it?
data[data.x0_Constellations == 1]


Constellations:  1
Cluster:  234
Cluster:  2241


,Year,Processor Speed (MHz),Total Cores,Co-Processor Cores to Total Cores,Date,Log(Rmax),x0_MPP,x0_Constellations,x1_AMD Zen-2 (Rome),x1_AMD Zen-3 (Milan),...,x1_SPARC64 VIIIfx,x1_SPARC64 XIfx,x1_SW-1,x1_SW-2,x1_SW-3,x1_SW26010,x1_SX-Aurora,x1_Unknown,x1_Vulcan,x1_Zen
7958,-1.6,-3.0,-0.377596,0.0,201111,3.967,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
